<a href="https://colab.research.google.com/github/heidyfp/Aula4/blob/main/Soporte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
!pip install -U -q google-generativeai

In [35]:
from flask import Flask, render_template, request
import google.generativeai as genai
import numpy as np
import pandas as pd

genai.configure(api_key='YOUR_API_KEY')
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('SECRET_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

app = Flask("Excel")

In [36]:
#Listando Documentos que seran buscados
DOCUMENT1 = {
    "title": "¿Por qué deberíamos aprender Excel?",
    "content": "La necesidad de aprender Excel se da principalmente para la gran mayoría de las personas porque es una exigencia para encontrar trabajo o porque es el programa, al que más fácil podemos acceder; sin necesidad de recurrir a un programador o alguien con conocimientos avanzados para gestionar nuestras propias bases de datos, sea para un negocio autónomo o para alguien que ya está trabajandoEn ese sentido podemos decir que, el que busca trabajo quiere aprender Excel porque es un requisito de la gran mayoría de empresas para conseguir el puesto o la vacante. Y el que ya trabaja, quiere o necesita aprender Excel porque mantiene alcanzado de trabajo y ve la necesidad de ser más rápido y eficiente con las tareas asignadas. Ahora, por parte del empleador, que sus empleados o colaboradores aprendan a usar Excel, se traduce en eficiencia y más calidad de la información que manejan. Y si eres autónomo, aprender Excel te permitirá gestionar tu empresa o negocio. Y para él qué es estudiante, requiere aprender Excel porque el sistema educativo sabe que es el programa más usado en todas las empresas y saber usar Excel o no, puede marcar la diferencia para insertarse a la vida laboral o para tener una mejor posición y un mejor sueldo."}

DOCUMENT2 = {
    "title": "¿Cómo aprender a usar Excel?",
    "content": "Teniendo en cuenta la versatilidad de Excel y sus ventajas, aprender Excel debería ser una de nuestras prioridades si queremos optimizar y ser más eficientes en nuestros trabajos, o en nuestras pruebas de ingreso con el fin de aplicar a una nueva vacante.Pese a que aprender a usar Excel puede parecer complicado, en realidad es algo muy sencillo y la mejor forma de aprender Excel paso a paso es mediante la práctica. Ir consiguiendo dominar cada una de las funciones de Excel y obtener una perspectiva orientada a cuando usar X o Y formula de Excel nos permitirá obtener resultados diferentes y encaminados a volvernos unos expertos en el manejo de la herramienta de Excel."}

DOCUMENT3 = {
    "title": "Aprender Excel paso a paso",
    "content": "Para aprender Excel paso a paso, es recomendable tomar un curso de Excel que te guíe por cada uno de sus niveles o si ya tienes conocimientos previos, validar el nivel en el que te encuentras para continuar con tu proceso de formación. Sin embargo, si puedes reforzar tus conocimientos en los conceptos básicos puedes encontrar cosas muy interesantes, sobre todo si lo que sabes de Excel ha sido obtenido de manera empírica.Con el curso de Excel podrás aprender de manera fácil y práctica todas las funcionalidades de las hojas de cálculo. Además, te guiará en la toma de decisiones que debes implementar al tratar de resolver algún caso en específico. Conocerás qué herramientas, funciones o fórmulas de Excel puedes utilizar para brindar solución a los problemas que se te presente o para tratar la información o base de datos que vayas a trabajar.La hoja de cálculo de Excel, al ser una herramienta tan personalizable, cada empresa le da el uso que cree conveniente y puede existir una gran diferencia entre estas. Por lo cual, aprender a usar Excel te permitirá comprender, interpretar y trabajar cualquier hoja de cálculo que haya sido trabajada anteriormente o crear tus propios sistemas o plantillas de trabajo.Si ¿Necesitas aprender Excel? nuestro curso de Excel está diseñado para que aprendas Excel de manera fácil y práctica. Conseguirás dominar Excel paso a paso y mediante las guías que te proporcionaremos podrás fortalecer tus habilidades y conseguir mejores resultados tanto en tu vida personal, académica, laboral y/o profesional.En definitiva, si sabes manejar la herramienta de Excel tienes muchas más facilidades de encontrar un nuevo trabajo, conseguir el ascenso o promoción que tanto quieres o gestionar eficientemente cualquier negocio."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]



In [37]:
model = "models/embedding-001"
def embed_fn(title,text, model):
  return genai.embed_content(model=model,
                                      content=text,
                                      title=title,
                                      task_type = "RETRIEVAL_DOCUMENT")['embedding']


In [38]:
df = pd.DataFrame(documents)
df.columns = ["Título","Conteudo"]
df

,Título,Conteudo
0,¿Por qué deberíamos aprender Excel?,La necesidad de aprender Excel se da principal...
1,¿Cómo aprender a usar Excel?,Teniendo en cuenta la versatilidad de Excel y ...
2,Aprender Excel paso a paso,"Para aprender Excel paso a paso, es recomendab..."


In [39]:
df["Embeddings"] = df.apply(lambda row:embed_fn(row["Título"], row["Conteudo"], model),axis=1)

In [40]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                      content=consulta,
                                      task_type = "RETRIEVAL_QUERY")
  productos_escalares = np.dot(np.stack(df['Embeddings']), embedding_da_consulta["embedding"])
  idice = np.argmax(productos_escalares)
  return df.iloc[idice]['Conteudo'] # Return text from index with max value

In [42]:
#consulta = 'Porque debo aprender excel'
#trecho = gerar_e_buscar_consulta(consulta, df, model)


In [43]:
generation_config = {
    "candidate_count": 1,
    "temperature": 0,
}

safety_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE",
}

In [44]:
model_2 = genai.GenerativeModel('gemini-1.0-pro', generation_config=generation_config,safety_settings=safety_settings)

In [45]:
chat= model_2.start_chat(history=[])

In [46]:
prompt = input('Hola, en que podemos ayudarle: ')

while prompt!= "fin":
  trecho = gerar_e_buscar_consulta(prompt, df, model)
  prompt = f'Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que no façam  parte do texto: {trecho}'
  model_2 = genai.GenerativeModel('gemini-1.0-pro', generation_config=generation_config,safety_settings=safety_settings)
  response = model_2.generate_content(prompt)
  print(response.text, '\n')
  prompt = input('Hola, en que podemos ayudarle: ')

Hola, en que podemos ayudarle: porque aprender excel
**Por que você precisa dominar o Excel:**

* **Pra arrumar trampo:** Muita empresa pede pra você saber Excel, então é tipo um passaporte pro mercado de trabalho.
* **Pra quem já tá na correria:** Excel te ajuda a ser mais rápido e eficiente nas suas tarefas, tipo um turbo pra sua produtividade.
* **Pra quem é chefe:** Ter funcionários que manjam de Excel é como ter uma equipe de super-heróis da informação.
* **Pra quem é autônomo:** Excel é o seu braço direito pra gerenciar o seu negócio.
* **Pra quem tá estudando:** O sistema educacional sabe que Excel é o cara nas empresas, então saber usar pode te dar uma vantagem na hora de entrar no mercado de trabalho ou até mesmo ganhar um salário melhor. 

Hola, en que podemos ayudarle: fin
